In [1]:
import control_utils.dynamixel_controller_pkg.dynamixel_controller as dc
import control_utils.dynamixel_controller_pkg.dynamixel_models as dm
import time
import numpy as np
import matplotlib.pyplot as plt
# plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "8"
from scipy import signal

In [2]:
nums = 24
motors= [dm.XM430W210(i) for i in range(nums)]

h = dc.DynamixelController("/dev/ttyUSB0", motors)

h.activate_controller()

h.torque_off()

# dynamixel_controller.set_operating_mode_all("position_control")
h.set_operating_mode_all("current_based_position_control")
# h.set_operating_mode_all("position_control")

h.torque_on()
q0 = np.zeros(nums)
q1 = np.ones(nums) * 0.5


In [3]:
h.set_position_p_gain([1500]*nums)
h.set_position_d_gain([2000]*nums)
# h.set_profile_velocity_rad([1, 1,1,1]*6)
h.set_goal_position_rad(q0)

current_limit = np.round(500 * np.ones(nums)).astype(int)
h.set_goal_current(current_limit)

h.set_return_delay_time(np.zeros(nums))

0

In [4]:
q1 = np.zeros(nums)
q1[0] = 0.5
h.set_goal_position_rad(q1)

0

In [4]:
h.read_info_with_unit(fast_read=True,retry=False)[0]  # position_list, velocity_list, current_list, pwm_list

array([ 0.00306797, -0.00920392, -0.00766994, -0.00613595,  0.00920392,
       -0.00920392, -0.01073791, -0.00613595,  0.00460196,  0.0122719 ,
        0.00613595,  0.00613595,  0.00766994,  0.01073791,  0.00920392,
        0.00460196,  0.        ,  0.01073791,  0.00766994,  0.00153399,
       -0.00766994,  0.00613595, -0.00613595, -0.00460196])

In [5]:

q0 = np.zeros(nums)
t0 = time.time()
ref='sin'
# ref='tri'
i = 0
f = 0.5
# a = np.array([0.4, 0.4,0.4,0.4]*5)
a = np.array([0.4, 0.4,0.4,0.4])
q_records = []
while 1:
    q_cmd = np.copy(q0)
    vel_cmd = np.zeros(nums)
    acc_cmd = np.zeros(nums)
    t = time.time() - t0
    if ref =='sin':
        q_i = a * np.sin(2 * np.pi *f *t)
        vel_cmd[:len(a)] = a * np.cos(2 * np.pi *f *t) * 2 * np.pi * f 
        acc_cmd[:len(a)] =  - a * np.sin(2 * np.pi *f *t) * (2 * np.pi * f )**2
    else:
        q_i = a * signal.sawtooth(2 * np.pi * f * (t + 1/f/4), 0.5)
        q_i = np.clip(q_i, -0.2,0.2)
    q_cmd[i: i+len(a)] = q_i
    # h.set_profile_velocity_rad(np.abs(vel_cmd) * 1 )
    # h.set_profile_acceleration(acc_cmd)
    h.set_goal_position_rad(q_cmd)
    # leap_hand.move_to(q_cmd)
    q_real, _, q_cur,_ = h.read_info_with_unit(fast_read=True,retry=False)
    # # q_records.append(np.array([t, q_i, q_real[i: i+len(a)]]))
    q_records.append(np.concatenate([np.array([t]), q_i, q_real[i: i+len(a)], q_cur[i: i+len(a)]]))
    # q_records.append(np.array([t, q_i, 0]))
    # q_records.append(t)
    # time.sleep(0.01)
    if t > 5 * 1/f:
        break
q_records = np.vstack(q_records)

In [6]:

fig_width = 90/25.4  * 0.7
import matplotlib
matplotlib.use('Agg')


error = []
for i in range(1,  1+ len(a)):
    fig, ax = plt.subplots(figsize=(fig_width, fig_width))
    ax.plot(q_records[:,0], q_records[:,i], color='r',label='Ref')
    ax.plot(q_records[:,0], q_records[:,i+len(a)], color='b',label='Real')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Joint position (rad)')
    ax.legend(loc='upper right')
    
    ax.set_title('Tracking performance test, joint ' + str(i))
    fig.savefig(  str(i) + '_joint_test.jpg',format='jpg', bbox_inches='tight',  pad_inches=0.0, dpi=600)
    
    print(str(i), 'mean tracking error:',   np.mean(np.abs(q_records[:,i] - q_records[:,i+len(a)]))/a[i-1])
    error.append(np.mean(np.abs(q_records[:,i] - q_records[:,i+len(a)]))/a[i-1])
print(np.mean(error))

1 mean tracking error: 0.03130550935739846
2 mean tracking error: 0.04266053626842422
3 mean tracking error: 0.02991479327775417
4 mean tracking error: 0.024992947219387166
0.032218446530741005


In [7]:
dt = q_records[1:,0] - q_records[:-1,0]

fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.plot(dt, color='r')
np.mean(dt)

0.01033803943760139

In [9]:
for i in range(4):
    print(np.std(np.abs(q_records[:, 1+ 2*len(a) + i])))
#     38.163911290429496
# 65.3932184500453
# 39.77045753633975
# 30.2007272325899

62.89400963606715
99.15201214691012
53.95446082170339
36.89820643476676
